In [ ]:
import sys
sys.path.append('../')
from src.kiss_data import KissRawData

In [ ]:
kd = KissRawData('/data/KISS/Raw/nika2c-data3/KISS/X20190301_1045_S0137_Moon_SCIENCEMAP')
kd

In [ ]:
len(kd)

In [ ]:
kd.names

In [ ]:
kd.header

In [ ]:
kd.kidpar

In [ ]:
kd.read_data()

In [ ]:
kd.calib_raw()

In [ ]:
fig = kd.calib_plot()
fig.savefig('calib.pdf')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
kd.kidfreq[:,:,0:135] = 0
for _kidfreq, name in zip(kd.kidfreq[0:5], kd.names.RawDataDetector[0:5]):
    plt.plot(_kidfreq[0], alpha=0.5, label=name)
plt.legend()

In [ ]:
flat = kd.kidfreq.reshape(kd.ndet, -1)
plt.imshow(np.corrcoef(flat[:, 0:1024*100]))
plt.colorbar()

In [ ]:
C_laser1_pos = kd.C_laser1_pos.reshape(-1, 1024)
for pos in C_laser1_pos[0:10]:
    _ = plt.plot(pos)

In [ ]:
plt.imshow(C_laser1_pos-C_laser1_pos.mean(axis=0), aspect='auto')
plt.colorbar()